# 1. Language Modeling

In this part, let's generate text using a trigram language model.

Go to https://drive.google.com/drive/folders/1pR0koayRSgXfTD72HZUHN14uec0SrnXy?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>


Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=False)

Mounted at /content/drive/


When you run the `ls` command below, you should see these folders.




In [ ]:
!ls "/content/drive/MyDrive/nl2ds"

semantic-parser  tweets


Let's load the trigrams first. You can change the below code as you see fit.

In [ ]:
from math import log

bigram_prefix_to_trigram = {}
bigram_prefix_to_trigram_weights = {}

lines = open("/content/drive/MyDrive/nl2ds/tweets/covid-tweets-2020-08-10-2020-08-21.trigrams.txt").readlines()
for line in lines:
  word1, word2, word3, count = line.strip().split()
  if (word1, word2) not in bigram_prefix_to_trigram:
    bigram_prefix_to_trigram[(word1, word2)] = []
    bigram_prefix_to_trigram_weights[(word1, word2)] = []
  bigram_prefix_to_trigram[(word1, word2)].append(word3)
  bigram_prefix_to_trigram_weights[(word1, word2)].append(int(count))

# freeup memory
lines = None

In [ ]:
items = {k: bigram_prefix_to_trigram[k] for k in list(bigram_prefix_to_trigram)[:100]}

## Problem 1.1: Retrieve top next words and their probability given a bigram prefix.

For the following prefixes **word1=middle, word2=of, and n=10**, the output is:



```
a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
...
...
...
```



In [ ]:
def top_next_word(word1, word2, n=10):
  # write your code here
  next_words = bigram_prefix_to_trigram[(word1,word2)]
  counts = bigram_prefix_to_trigram_weights[(word1,word2)]
  probabilities = counts/np.max(counts)

  arr = list(zip(next_words,probabilities))
  arr = np.array(arr)[0:n]
  return arr[:,0],arr[:,1]


next_words, probs = top_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 1.0
the 0.08599651365485184
pandemic 0.029633933759442184
this 0.020918070889018012
an 0.01336432306798373
covid 0.011621150493898896
nowhere 0.010459035444509006
it 0.005810575246949448
lockdown 0.002905287623474724
summer 0.002905287623474724


## Problem 1.2: Sampling n words

Sample next n words given a bigram prefix. Use the probablity distribution defined by the frequency counts. Functions like **numpy.random.choice** will be useful here. Sample without repitition, otherwise all your samples will contain the most frequent trigram.


For the following prefixes **word1=middle, word2=of, and n=10**, the output could be as follows (our outputs may differ): 

```
a 0.807981220657277
pandemic 0.023943661971830985
nowhere 0.008450704225352112
the 0.06948356807511737
...
...
...
...
...
```



In [ ]:
import random
def sample_next_word(word1, word2, n=10):
  next_words = bigram_prefix_to_trigram[(word1,word2)]
  counts = bigram_prefix_to_trigram_weights[(word1,word2)]
  probabilities = counts/np.max(counts)

  arr = list(zip(next_words,probabilities))
  # Get top n probabilities
  # arr =sorted(arr,reverse=True, key=lambda x: x[1])
  random.shuffle(arr)
  arr = np.array(arr)[0:n]
  return arr[:,0],arr[:,1]


next_words, probs = sample_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

deadly 0.0011621150493898896
endorsement 0.0005810575246949448
year 0.0005810575246949448
it 0.005810575246949448
URL 0.0005810575246949448
summer 0.002905287623474724
living 0.0005810575246949448
stage 0.002324230098779779
freaking 0.0005810575246949448
‘ 0.0005810575246949448


## Problem 1.3: Generate sentences starting with a prefix

Generates n-sentences starting with a given sentence prefix. Use [beam search](https://en.wikipedia.org/wiki/Beam_search) to generate multiple sentences. Depending on which method you use to generate next word, you will get different outputs. When you generate <EOS> in a path, stop exploring that path. If you are not careful with your implementation, you may end up in an infinite loop.

If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output is as follows:
```
<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.0001719607222046247
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
...
...
...
```


If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output is as follows:
```
<BOS1> <BOS2> biden calls for a 30 bonus URL #cashgem #cashappfriday #stayathome <EOS> 0.0002495268686322749
<BOS1> <BOS2> biden says all u.s. governors should mandate masks <EOS> 1.6894510541025754e-05
<BOS1> <BOS2> biden says all u.s. governors question cost of a pandemic <EOS> 8.777606198953028e-07
...
...
...
```


If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output may look as follows (since this is sampling, our outputs will difer):

```
<BOS1> <BOS2> trump signs executive orders URL <EOS> 7.150992253427233e-05
<BOS1> <BOS2> trump signs executive actions URL <EOS> 7.117242889600614e-05
<BOS1> <BOS2> trump news president attacked over it <EOS> 1.0546494007903964e-05
<BOS1> <BOS2> trump news president attacked over executive orders URL <EOS> 1.0126405114118984e-05
```

If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output may look as follows:

```
<BOS1> <BOS2> biden harris 2020 <EOS> 0.0015758924114719264
<BOS1> <BOS2> biden harris 2020 URL <EOS> 0.0006443960952032196
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do it URL <EOS> 4.105215709355001e-07
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do our best to stay home <EOS> 1.3158806336098573e-09
...
...
...
...
...
```

Hope you see that sampling gives different outputs compared to deterministically picking the top n-words.


In [ ]:
def generate_sentences(prefix, beam, sampler):
  sentences = []
  sents_probs = []
  sentence = ''
  counter = {}

  for i in range(beam):
    if not sentence: sentence =  prefix.split()
    probs = []
    w1,w2 = sentence[-2:]
    if ((w1,w2) in counter.keys()) and (counter[(w1,w2)] > 0 or complete):
      sentence = sentence[0:-2]
      w1,w2 = sentence[-2:]
    while sentence[-1] != '<EOS>' or (w1,w2) in bigram_prefix_to_trigram:
        counter[(w1,w2)] = counter[(w1,w2)]+ 1 if (w1,w2) in counter.keys() else 0
        w3,prob = sampler(w1,w2)
        # print(counter)
        if counter[(w1,w2)] < len(w3):
          complete = False
          sentence.append(w3[counter[(w1,w2)]])
          probs.append(float(prob[counter[(w1,w2)]]))
        else:
          sentence = sentence[0:-2]
          complete = True

        w1,w2 = sentence[-2:]
    sentences.append(' '.join(map(str,sentence)))
    sents_probs.append(sum(probs)/len(probs))
    # print(counter)
    
    sentence = sentence[0:-2]
  
  return sentences,sents_probs
  

  # write your code



sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)

<BOS1> <BOS2> trump is a pandemic <EOS> 1.0
<BOS1> <BOS2> trump is a hoax and that ’s why i do n’t have to wear a mask and social distancing and masks are not the same time <EOS> 0.9965635738831615
<BOS1> <BOS2> trump is a hoax and that ’s why i do n’t have to wear a mask and social distancing and masks are not the same thing URL <EOS> 0.9158780231335436
<BOS1> <BOS2> trump is a hoax and that ’s why i do n’t have to wear a mask and social distancing and masks are not the same thing as a result of the pandemic <EOS> 0.9841269841269842
<BOS1> <BOS2> trump is a hoax and that ’s why i do n’t have to wear a mask and social distancing and masks are not the same thing as a result of the world is going to be a good idea to go to the pandemic is over <EOS> 0.954134884673644
<BOS1> <BOS2> trump is a hoax and that ’s why i do n’t have to wear a mask and social distancing and masks are not the same thing as a result of the world is going to be a good idea to go to the pandemic is a good thing abou

# 2. Semantic Parsing

In this part, you are going to build your own virtual assistant! We will be developing two modules: an intent classifier and a slot filler.

In [ ]:
!ls "/content/drive/MyDrive/nl2ds/semantic-parser"
parser_files = "/content/drive/MyDrive/nl2ds/semantic-parser"

test_answers.txt  test_questions.txt  train_questions_answers.txt


In [ ]:
import json

train_data = []
for line in open(f'{parser_files}/train_questions_answers.txt'):
    train_data.append(json.loads(line))

# print a few examples
for i in range(5):
    print(train_data[i])
    print("-"*80)

{'question': 'Add an album to my Sylvia Plath playlist.', 'intent': 'AddToPlaylist', 'slots': {'music_item': 'album', 'playlist_owner': 'my', 'playlist': 'Sylvia Plath'}}
--------------------------------------------------------------------------------
{'question': 'add Diarios de Bicicleta to my la la playlist', 'intent': 'AddToPlaylist', 'slots': {'playlist': 'Diarios de Bicicleta', 'playlist_owner': 'my', 'entity_name': 'la la'}}
--------------------------------------------------------------------------------
{'question': 'book a table at a restaurant in Lucerne Valley that serves chicken nugget', 'intent': 'BookRestaurant', 'slots': {'restaurant_type': 'restaurant', 'city': 'Lucerne Valley', 'served_dish': 'chicken nugget'}}
--------------------------------------------------------------------------------
{'question': 'add iemand als jij to my playlist named In The Name Of Blues', 'intent': 'AddToPlaylist', 'slots': {'entity_name': 'iemand als jij', 'playlist_owner': 'my', 'playlist'

In [ ]:
test_questions = []
for line in open(f'{parser_files}/test_questions.txt'):
    test_questions.append(json.loads(line))

test_answers = []
for line in open(f'{parser_files}/test_answers.txt'):
    test_answers.append(json.loads(line))

# print a few examples
for i in range(5):
    print(test_questions[i])
    print(test_answers[i])
    print("-"*80)

Add an artist to Jukebox Boogie Rhythm & Blues
{'intent': 'AddToPlaylist', 'slots': {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}}
--------------------------------------------------------------------------------
Will it be rainy at Sunrise in Ramey Saudi Arabia?
{'intent': 'GetWeather', 'slots': {'condition_description': 'rainy', 'timeRange': 'Sunrise', 'city': 'Ramey', 'country': 'Saudi Arabia'}}
--------------------------------------------------------------------------------
Weather in two hours  in Uzbekistan
{'intent': 'GetWeather', 'slots': {'timeRange': 'in two hours', 'country': 'Uzbekistan'}}
--------------------------------------------------------------------------------
Will there be a cloud in VI in 14 minutes ?
{'intent': 'GetWeather', 'slots': {'condition_description': 'cloud', 'state': 'VI', 'timeRange': 'in 14 minutes'}}
--------------------------------------------------------------------------------
add nuba to my Metal Party playlist
{'intent': 

## Problem 2.1: Keyword-based intent classifier

In this part, you will build a keyword-based intent classifier. For each intent, come up with a list of keywords that are important for that intent, and then classify a given question into an intent. If an input question matches multiple intents, pick the best one. If it does not match any keyword, return None.

Caution: You are allowed to look at training questions and answers to come up with a set of keywords, but it is a bad practice to look at test answers. 

In [ ]:
# List of all intents
intents = set()
for example in train_data:
    intents.add(example['intent'])
print(intents)

{'AddToPlaylist', 'BookRestaurant', 'GetWeather'}


In [ ]:
!pip install rake-nltk

     |████████████████████████████████| 1.5 MB 5.7 MB/s 
     |████████████████████████████████| 749 kB 46.4 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def getKeywords(text):
  rake_nltk_var = Rake()
  rake_nltk_var.extract_keywords_from_text(text)
  keywords = rake_nltk_var.get_ranked_phrases()
  return keywords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
BookRestaurant, AddToPlaylist, GetWeather =({},{},{})
for train in train_data:
  keywords = getKeywords(train['question'])
  for keyword in keywords:
    if train['intent'] == 'BookRestaurant':
      BookRestaurant[keyword] = BookRestaurant[keyword]+1 if keyword in BookRestaurant.keys() else 1
    elif train['intent'] == 'AddToPlaylist':
      AddToPlaylist[keyword] = AddToPlaylist[keyword]+1 if keyword in AddToPlaylist.keys() else 1
    elif train['intent'] == 'GetWeather':
      GetWeather[keyword] = GetWeather[keyword]+1 if keyword in GetWeather.keys() else 1

In [ ]:
def predict_intent_using_keywords(question):
  # Fill in your code here.
  keywords = getKeywords(question)
  result_dic = {"BookRestaurant":[],"AddToPlaylist":[],"GetWeather":[]}
  for keyword in keywords:
    result_dic['BookRestaurant'].append({'keyword':keyword,'count':BookRestaurant[keyword] if keyword in BookRestaurant.keys() else 0})
    result_dic['AddToPlaylist'].append({'keyword':keyword,'count':AddToPlaylist[keyword] if keyword in AddToPlaylist.keys() else 0})
    result_dic['GetWeather'].append({'keyword':keyword,'count':GetWeather[keyword] if keyword in GetWeather.keys() else 0})
  best_intent = {"BookRestaurant":0,"AddToPlaylist":0,"GetWeather":0}
  for intent in best_intent.keys():
    for item in result_dic[intent]:
      best_intent[intent] += item['count']
  best_intent = max(best_intent,key=best_intent.get)
  return best_intent

In [ ]:
from collections import Counter

'''Gives intent wise accuracy of your model'''
def evaluate_intent_accuracy(prediction_function_name):
  correct = Counter()
  total = Counter()
  for i in range(len(test_questions)):
    q = test_questions[i]
    gold_intent = test_answers[i]['intent']
    predict_intent = prediction_function_name(q)
    if prediction_function_name(q) == gold_intent:
      correct[gold_intent] += 1
    total[gold_intent] += 1
  for intent in intents:
    print(intent, correct[intent]/total[intent], total[intent])
    
# Evaluating the intent classifier. 
# In our implementation, a simple keyword based classifier has achieved an accuracy of greater than 65 for each intent
evaluate_intent_accuracy(predict_intent_using_keywords)

AddToPlaylist 0.8 100
BookRestaurant 1.0 100
GetWeather 0.9 100


## Problem 2.2: Statistical intent classifier

Now, let's build a statistical intent classifier. Instead of making use of keywords like what you did above, you will first extract features from a given input question. In order to build a feature representation for a given sentence, make use of word2vec embeddings of each word and take an average to represent the sentence. Then train a logistic regression. Feel free to use any libraries you like.

In [ ]:
import nltk
nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [ ]:
from nltk.data import find
import gensim

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
len(word2vec_model['question'])
len(word2vec_model['answer'])


300

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
stop = set(stopwords.words('english') + list(string.punctuation))

'''Trains a logistic regression model on the entire training data. For an input question (x), the model learns to predict an intent (Y).'''
def train_logistic_regression_intent_classifier():
    global stop
    # Fill in your code here
    # Feel free to add more cells or functions if needed
    X,y = ([],[])
    for sdata in train_data:
      imp_keywords = [i for i in word_tokenize(sdata['question'].lower()) if i not in stop]
      all_arr = []
      for key in imp_keywords:
        if key in word2vec_model:
          all_arr.append(word2vec_model[key])
      all_arr = np.array(all_arr)
      avg_arr = all_arr.mean(axis=0) 
      if not isinstance(avg_arr,np.ndarray):
        avg_arr = [0]*300
      # print(avg_arr.shape)
      X.append(avg_arr)
      y.append(sdata['intent'])
    # label_encoder = LabelEncoder()
    # integer_encoded = label_encoder.fit_transform(y)
    y = np.array(y)
    y = y.reshape(-1,1)
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    X = np.array(X)
    X = np.nan_to_num(X)
    model.fit(X,y)
    return model

In [ ]:
'''For an input question, the model predicts an intent'''
def predict_intent_using_logistic_regression(model,question):
    # Fill in your code here
    # Feel free to add more cells or functions if needed
    imp_keywords = [i for i in word_tokenize(question.lower()) if i not in stop]
    avg_arr = []
    for key in imp_keywords:
      if key in word2vec_model.wv:
        avg_arr.append(word2vec_model[key])
    avg_arr = np.array(avg_arr)
    predictions = model.predict(np.array([avg_arr.mean(axis=0)]))
    return predictions[0]

In [ ]:
from collections import Counter

'''Gives intent wise accuracy of your model'''
def evaluate_intent_accuracy_regression(prediction_function_name):
  model = train_logistic_regression_intent_classifier()

  correct = Counter()
  total = Counter()
  for i in range(len(test_questions)):
    q = test_questions[i]
    gold_intent = test_answers[i]['intent']
    predict_intent = prediction_function_name(model,q)
    if predict_intent == gold_intent:
      correct[gold_intent] += 1
    total[gold_intent] += 1
  print(intents,'==='*80)
  for intent in intents:
    print(intent, correct[intent]/total[intent], total[intent])
    

In [ ]:
# Evaluate the intent classifier
# Your intent classifier performance will be close to 100 if you have done a good job.
evaluate_intent_accuracy_regression(predict_intent_using_logistic_regression)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4

{'AddToPlaylist', 'BookRestaurant', 'GetWeather'} ================================================================================================================================================================================================================================================
AddToPlaylist 1.0 100
BookRestaurant 1.0 100
GetWeather 0.99 100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
 

## Problem 2.3: Slot filling

Build a slot filling model. We will just work with `AddToPlaylist` intent. Ignore other intents.

Hint: No need to rely on machine learning here. You can use ideas like maximum string matching to identify which slots are active and what thier values are. This problem's solution is intentionally left underspecified.

In [ ]:
# Let's stick to one target intent.
target_intent = "AddToPlaylist"

# This intent has the following slots
target_intent_slot_names = set()
for sample in train_data:
    if sample['intent'] == target_intent:
        for slot_name in sample['slots']:
            target_intent_slot_names.add(slot_name)
print(target_intent_slot_names)


# Extract all the relevant questions of this target intent from the test examples.
target_intent_questions = []
target_intent_answers = []
for i, question in enumerate(test_questions):
    if test_answers[i]['intent'] == target_intent:
        target_intent_questions.append(question)
        target_intent_answers.append(test_answers[i])
print(len(target_intent_questions))

{'entity_name', 'playlist', 'artist', 'playlist_owner', 'music_item'}
100


In [ ]:
def initialize_slots():
    slots = {}
    for slot_name in target_intent_slot_names:
        slots[slot_name] = None
    return slots

def predict_slot_values(question):
    slots = initialize_slots()
    tslots = {}
    for data in train_data:
      if data['intent'] == 'AddToPlaylist':
        test_keywords = getKeywords(question)
        train_keywords = getKeywords(data['question'])
        count = 0
        for tkey in train_keywords:
          if tkey in test_keywords:
            count += 1
        if count >= int((len(test_keywords)+len(train_keywords))/2)-1:
          tslots = data['slots']
          break
    for slot_name in target_intent_slot_names:
        # Fill in your code to idenfity the slot value. By default, they are initialized to None.
        if slot_name in tslots.keys():
          slots[slot_name] = tslots[slot_name]
    return slots

def evaluate_slot_prediction_recall(slot_prediction_function):
    correct = Counter()
    total = Counter()
    # predict slots for each question
    for i, question in enumerate(target_intent_questions):
        gold_slots = test_answers[i]['slots']
        predicted_slots = slot_prediction_function(question)
        for name in target_intent_slot_names:
            if name in gold_slots:
                total[name] += 1.0
                if predicted_slots.get(name, None) == gold_slots.get(name):
                    correct[name] += 1.0
    for name in target_intent_slot_names:
        print(f"{name}: {correct[name] / total[name]}")


# Our reference implementation got these numbers. You can ask others on Slack what they got.
# music_item 1.0
# playlist 0.67
# artist  0.021739130434782608
# playlist_owner 0.9444444444444444
# entity_name 0.1111111111111111
print("Slot accuracy for your slot prediction model")
evaluate_slot_prediction_recall(predict_slot_values)


Slot accuracy for your slot prediction model
entity_name: 0.0
playlist: 0.0
artist: 0.0
playlist_owner: 0.4117647058823529
music_item: 0.2857142857142857


In [ ]:
# Find a true positive prediction for each slot
# Fill in your code below along with print statement
correct = Counter()
total = Counter()
# predict slots for each question
for i, question in enumerate(target_intent_questions):
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    for name in target_intent_slot_names:
        if name in gold_slots:
            total[name] += 1.0
            if (predicted_slots[name] is not None) and (predicted_slots.get(name, None) == gold_slots.get(name)):
                correct[name] += 1.0
for name in target_intent_slot_names:
    print(f"{name}: {correct[name] / total[name]}")


entity_name: 0.0
playlist: 0.0
artist: 0.0
playlist_owner: 0.4117647058823529
music_item: 0.2857142857142857


In [ ]:
question = target_intent_questions[0]
predicted_slots = predict_slot_values(question)
gold_slots = test_answers[0]['slots']
print(predicted_slots)
print(gold_slots)
print(predicted_slots.get('entity_name',None))

{'entity_name': None, 'playlist': '59th grammy awards', 'artist': None, 'playlist_owner': 'my', 'music_item': 'artist'}
{'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
None


In [ ]:
# Find a false positive prediction for each slot
# Fill in your code below along with print statement

correct = Counter()
total = Counter()
# predict slots for each question
for i, question in enumerate(target_intent_questions):
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    for name in target_intent_slot_names:
        if name in gold_slots:
            total[name] += 1.0
            if (predicted_slots[name] is not None) and (predicted_slots.get(name, None) != gold_slots.get(name)):
                correct[name] += 1.0
for name in target_intent_slot_names:
    print(f"{name}: {correct[name] / total[name]}")



entity_name: 0.0
playlist: 0.8235294117647058
artist: 0.07142857142857142
playlist_owner: 0.0
music_item: 0.42857142857142855


In [ ]:
# Find a true negative prediction for each slot
# Fill in your code below along with a print statement
correct = Counter()
total = Counter()
# predict slots for each question
for i, question in enumerate(target_intent_questions):
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    for name in target_intent_slot_names:
        if name in gold_slots:
            total[name] += 1.0
            if (predicted_slots[name] is None) and (predicted_slots.get(name, None) == gold_slots.get(name)):
                correct[name] += 1.0
for name in target_intent_slot_names:
    print(f"{name}: {correct[name] / total[name]}")


entity_name: 0.0
playlist: 0.0
artist: 0.0
playlist_owner: 0.0
music_item: 0.0


In [ ]:
# Find a false negative prediction for each slot
# Fill in your code below along with a print statement

correct = Counter()
total = Counter()
# predict slots for each question
for i, question in enumerate(target_intent_questions):
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    for name in target_intent_slot_names:
        if name in gold_slots:
            total[name] += 1.0
            if (predicted_slots[name] is None) and (predicted_slots.get(name, None) != gold_slots.get(name)):
                correct[name] += 1.0
for name in target_intent_slot_names:
    print(f"{name}: {correct[name] / total[name]}")


entity_name: 1.0
playlist: 0.17647058823529413
artist: 0.9285714285714286
playlist_owner: 0.5882352941176471
music_item: 0.2857142857142857
